# We will demonstrate skpref on discrete choice data

We wil use the swissmetro dataset available to download on https://transp-or.epfl.ch/pythonbiogeme/examples_swissmetro.html
This dataset tracks 470 respondents on which transportation alternative they have taken. There are 3 options in general: train, car and swissmetro.
More details on the original use of the dataset can be found here: http://strc.ch/2001/bierlaire1.pdf

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import sys
sys.path.insert(0, "../..")
from skpref.base import ClassificationReducer
from skpref.random_utility import BradleyTerry
from skpref.task import ChoiceTask
from skpref.metrics import f1_score, log_loss, log_loss_compare_with_t_test
from skpref.utils import nice_print_results
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
from sklearn.linear_model import LogisticRegression

In [2]:
swissmetro = pd.read_csv("data/swissmetro.dat", sep='\t')
swissmetro.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,MALE,INCOME,GA,ORIGIN,DEST,TRAIN_AV,CAR_AV,SM_AV,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,36,60,63,42,20,0,90,84,2


Looking at the data we can see that each row represents a choice.
The full explanations of variables can be found here: https://transp-or.epfl.ch/pythonbiogeme/examples/swissmetro/swissmetro.pdf

# We will need to make a couple of changes to the format of this table for skpref to read it easier. <br>

1) In this table the availability of alternatives is marked by TRAIN_AV, CAR_AV, SM_AV which indicate with 1 if the alternative is available and 0 otherwise. We need to convert these to a column that contains a list of alternatives for each row.

2) The choices are indicated by the CHOICE column, which contains 0 for unknown (we will dropping these), 1 for Train, 2 for Swissmetro and 3 for a Car usage. We need to name these explicitly. We could just create a list of alternatives called 1,2,3 in step 1 which would bypass step 2, however, we prefer the clarity of having the alternatives named explicitly.

# We will use a subset of the available features to make the skpref demo easier
Under normal circumstances we would use one-hot encoding on a lot of the binary features before training a serious model, however, to make the demo simple, we will only use the travel time and cost features. Users can of course do whatever feature transformations they like before fitting a model.
To build a classifier based on travel time and costs, we first need to split some columns from the swissmetro dataset into a secondary table.

In [3]:
train_vals = swissmetro[['TRAIN_TT', 'TRAIN_CO']].copy()
train_vals.columns = ['Travel Time', 'Cost']
train_vals.reset_index(inplace=True)
train_vals['alternative'] = 'Train'
swissmetro_vals = swissmetro[['SM_TT', 'SM_CO']].copy()
swissmetro_vals.columns = ['Travel Time', 'Cost']
swissmetro_vals.reset_index(inplace=True)
swissmetro_vals['alternative'] = 'Swiss Metro'
car_vals = swissmetro[['CAR_TT', 'CAR_CO']].copy()
car_vals.columns = ['Travel Time', 'Cost']
car_vals.reset_index(inplace=True)
car_vals['alternative'] = 'Car'
dummy_secondary_table = (
    train_vals.append(swissmetro_vals.append(car_vals))
).sort_values('index')
dummy_secondary_table.rename(columns={'index': 'merge_index'}, inplace=True)
dummy_secondary_table.head()

,merge_index,Travel Time,Cost,alternative
0,0,112,48,Train
0,0,63,52,Swiss Metro
0,0,117,65,Car
1,1,103,48,Train
1,1,60,49,Swiss Metro


In [4]:
binary_concats = (swissmetro.TRAIN_AV.astype(str) + 
                  swissmetro.CAR_AV.astype(str)  + 
                  swissmetro.SM_AV.astype(str)
                 )

alts = []
for i in binary_concats.values:
    if i == '111':
        alts.append(['Train', 'Car', 'Swiss Metro'])
    elif i == '100':
        alts.append(['Train'])
    elif i == '000':
        alts.append(['None'])
    elif i == '010':
        alts.append(['Car'])
    elif i == '001':
        alts.append(['Swiss Metro'])
    elif i == '101':
        alts.append(['Train', 'Swiss Metro'])
    elif i == '110':
        alts.append(['Train', 'Car'])
    elif i == '011':
        alts.append(['Car', 'Swiss Metro'])
        
swissmetro['alternatives'] = alts
        
swissmetro['chosen'] = np.where(swissmetro.CHOICE.values==1, 'Train',
                       np.where(swissmetro.CHOICE.values==2, 'Swiss Metro',
                       np.where(swissmetro.CHOICE.values==3, 'Car',
                                'unknown')))
swissmetro = swissmetro[swissmetro.CHOICE != 0].copy()
swissmetro = swissmetro.reset_index()[['alternatives', 'chosen', 'index']]
swissmetro.rename(columns={'index': 'merge_index'}, inplace=True)
swissmetro.head()

,alternatives,chosen,merge_index
0,"[Train, Car, Swiss Metro]",Swiss Metro,0
1,"[Train, Car, Swiss Metro]",Swiss Metro,1
2,"[Train, Car, Swiss Metro]",Swiss Metro,2
3,"[Train, Car, Swiss Metro]",Swiss Metro,3
4,"[Train, Car, Swiss Metro]",Swiss Metro,4


In [5]:
swissmetro.alternatives.values

array([list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']), ...,
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro'])], dtype=object)

In [6]:
train, test = train_test_split(swissmetro, random_state=1, test_size=0.1)

swiss_metro_train = ChoiceTask(train, 'alternatives', 'chosen',
                               features_to_use=['Travel Time', 'Cost'],
                               secondary_table=dummy_secondary_table,
                               secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                              )

swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )

In [7]:
my_log_red = ClassificationReducer(LogisticRegression(solver='lbfgs'))
my_log_red.fit_task(swiss_metro_train)
preds = my_log_red.predict_proba_task(swiss_metro_test,
                                      ['Swiss Metro','Train', 'Car'])

In [8]:
nice_print_results(preds)

Swiss Metro  [0.49 0.56 0.4  ... 0.47 0.35 0.51]
Train        [0.3  0.45 0.17 ... 0.36 0.29 0.26]
Car          [0.33 0.   0.2  ... 0.45 0.36 0.25]


In [9]:
outocme_preds = my_log_red.predict_task(swiss_metro_test)
print(outocme_preds.top_input_data[:5])
print(outocme_preds.boot_input_data[:5])

['Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro']
[array(['Car', 'Train'], dtype='<U11') array(['Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')]


In [10]:
# Fit a Bradley Terry model with no features
swiss_metro_train_BT = ChoiceTask(train.drop('merge_index', axis=1), 
                                  'alternatives',
                                  'chosen', features_to_use=None)

swiss_metro_test_BT = ChoiceTask(test.drop('merge_index', axis=1), 
                                 'alternatives',
                                 'chosen', features_to_use=None)

In [11]:
my_BT_red = BradleyTerry(method='BFGS', alpha=1e-5)
my_BT_red.fit_task(swiss_metro_train_BT)
preds = my_BT_red.predict_proba_task(swiss_metro_test_BT,
                                     ['Swiss Metro','Train', 'Car'])

In [12]:
nice_print_results(preds)

Swiss Metro  [0.55 0.83 0.55 ... 0.55 0.55 0.55]
Train        [0.11 0.17 0.11 ... 0.11 0.11 0.11]
Car          [0.35 0.   0.35 ... 0.35 0.35 0.35]


In [13]:
choice_preds = my_BT_red.predict_task(swiss_metro_test_BT)
print(choice_preds.top_input_data[:5])
print(choice_preds.boot_input_data[:5])

['Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro']
[array(['Car', 'Train'], dtype='<U11') array(['Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')]


In [14]:
# Fit a Bradley Terry model with features
# Reducing training set because on my PC this gives a memory error
swiss_metro_train_BT_feats = ChoiceTask(
    train.sample(frac=0.5), 'alternatives', 'chosen',
    secondary_table=dummy_secondary_table,
    secondary_to_primary_link={
        'merge_index': 'merge_index',
        'alternative': 'alternatives'
    },
    features_to_use=['Travel Time', 'Cost'])

swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )

my_BT_red_feats = BradleyTerry(method='BFGS', alpha=1e-5)
my_BT_red_feats.fit_task(swiss_metro_train_BT_feats)

In [15]:
swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )
preds = my_BT_red_feats.predict_proba_task(swiss_metro_test,
                                      ['Swiss Metro','Train', 'Car'])

In [16]:
my_BT_red_feats.predict_task(swiss_metro_test).top_input_data

array(['Swiss Metro', 'Swiss Metro', 'Swiss Metro', ..., 'Car', 'Car',
       'Swiss Metro'], dtype=object)

In [17]:
nice_print_results(preds)

Swiss Metro  [0.58 0.76 0.71 ... 0.39 0.35 0.73]
Train        [0.11 0.24 0.07 ... 0.13 0.14 0.09]
Car          [0.3  0.   0.22 ... 0.49 0.5  0.18]


In [18]:
ind_trans_preds = my_BT_red_feats.predict_proba_task(swiss_metro_test,
                                   ['Swiss Metro','Train', 'Car'],
                                   aggregation_method='independent transitive')
nice_print_results(ind_trans_preds)

Swiss Metro  [0.65 0.76 0.78 ... 0.41 0.36 0.81]
Train        [0.05 0.24 0.03 ... 0.06 0.08 0.04]
Car          [0.3  0.   0.2  ... 0.53 0.56 0.15]


In [19]:
preds_outcome_ind_trans = my_BT_red_feats.predict_task(swiss_metro_test,
                             aggregation_method='independent transitive')
preds_outcome_Luce = my_BT_red_feats.predict_task(swiss_metro_test)

In [20]:
dummy_secondary_table.groupby('alternative').mean()

,merge_index,Travel Time,Cost
alternative,,,
Car,5363.5,123.795209,78.742077
Swiss Metro,5363.5,87.466350,670.340697
Train,5363.5,166.626025,514.335477


In [21]:
my_BT_red_feats.bt_with_feats.print_summaries()



Number of Parameters                                                      5
Number of Observations                                                 8919
Null Log-Likelihood                                            -6182.179703
Fitted Log-Likelihood                                          -4557.619058
Rho-Squared                                                        0.262781
Rho-Bar-Squared                                                    0.261972
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object
             parameters     std_err    t_stats       p_values  robust_std_err  \
Cost           0.000295    0.000033   9.072674   1.161315e-19        0.000033   
Travel Time   -0.014824    0.000590 -25.105296  4.353271e-139        0.000688   
Car            0.365526  129.099450   0.002831   9.977409e-01             NaN   
Swiss Metro    0.054123  129.099451   0.000419   9.996655e-01             NaN   
Train         -0.419649  129.099451  -0.003251 

In [22]:
print(f"The F1 score of the Luce aggregation was \
{f1_score(swiss_metro_test.subset_vec, preds_outcome_Luce): .3}")
print(f"The F1 score of the generic aggregation was \
{f1_score(swiss_metro_test.subset_vec, preds_outcome_ind_trans):.3}")

The F1 score of the Luce aggregation was  0.593
The F1 score of the generic aggregation was 0.593


In [23]:
random_probs = {
    'Swiss Metro': np.ones(len(test)) * (1/3),
    'Train': np.ones(len(test)) * (1/3),
    'Car': np.ones(len(test)) * (1/3)
}
print(f"The log loss for each alternative in the Luce aggregation was \n \
{log_loss(swiss_metro_test.subset_vec, preds)}")
print(f"The log loss for each alternative in the generic aggregation was \n \
{log_loss(swiss_metro_test.subset_vec, ind_trans_preds)}")
print(f"The log loss for each alternative assigning random probability was \n \
{log_loss(swiss_metro_test.subset_vec, random_probs)}")

The log loss for each alternative in the Luce aggregation was 
 {'Swiss Metro_log_loss': 0.71, 'Train_log_loss': 0.38, 'Car_log_loss': 0.53}
The log loss for each alternative in the generic aggregation was 
 {'Swiss Metro_log_loss': 0.73, 'Train_log_loss': 0.39, 'Car_log_loss': 0.53}
The log loss for each alternative assigning random probability was 
 {'Swiss Metro_log_loss': 0.8, 'Train_log_loss': 0.5, 'Car_log_loss': 0.61}


In [24]:
print(f"The t-test for H0: Luce aggregation = Generic aggregation \
{log_loss_compare_with_t_test(swiss_metro_test.subset_vec, preds, ind_trans_preds)}")
print(f"The t-test for H0: Generic aggregation = random probability \
{log_loss_compare_with_t_test(swiss_metro_test.subset_vec, ind_trans_preds, random_probs)}")

The t-test for H0: Luce aggregation = Generic aggregation {'Swiss Metro': 0.0, 'Train': 0.1, 'Car': 0.01}
The t-test for H0: Generic aggregation = random probability {'Swiss Metro': 0.01, 'Train': 0.0, 'Car': 0.0}


../..\skpref\metrics\_classification.py:254: RuntimeWarning: divide by zero encountered in log
  logged = np.log(predicted1[_alternative])
../..\skpref\metrics\_classification.py:258: RuntimeWarning: invalid value encountered in multiply
  np.nan_to_num(logged * binarized_outcome) +
../..\skpref\metrics\_classification.py:261: RuntimeWarning: divide by zero encountered in log
  logged2 = np.log(predicted2[_alternative])
../..\skpref\metrics\_classification.py:265: RuntimeWarning: invalid value encountered in multiply
  np.nan_to_num(logged2 * binarized_outcome) +
